## **Proyecto de Predicción de Churn en empresa de Telecomunicaciones**

# **1. Configuración del Ambiente**

In [2]:
#Se cargan todas las librerias necesarias para el proyecto

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

#biblioteca para balancear los datos utilizando over_sampling
from imblearn.over_sampling import SMOTE

In [3]:
#Se establece una variable global que contendrá el dataframe
global datos_churn

# **2. Obtención y Tratamiento de Datos**

## **2.1 Cargando las bases de datos**

In [4]:
#Se cargan los datos que están guardados en formato '.json'
datos_churn = pd.read_json("base_clientes.json")
datos_churn.head()

,id_cliente,Churn,cliente,telefono,internet,cuenta
0,0002-ORFBO,no,"{'genero': 'femenino', 'anciano': 0, 'pareja':...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'DSL', 'seguridad_online...","{'contrato': None, 'facturacion_electronica': ..."
1,0003-MKNFE,no,"{'genero': 'masculino', 'anciano': 0, 'pareja'...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'DSL', 'seguridad_online...","{'contrato': 'mensual', 'facturacion_electroni..."
2,0004-TLHLJ,si,"{'genero': 'masculino', 'anciano': 0, 'pareja'...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'fibra optica', 'segurid...","{'contrato': 'mensual', 'facturacion_electroni..."
3,0011-IGKFF,si,"{'genero': 'masculino', 'anciano': 1, 'pareja'...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'fibra optica', 'segurid...","{'contrato': 'mensual', 'facturacion_electroni..."
4,0013-EXCHZ,si,"{'genero': 'femenino', 'anciano': 1, 'pareja':...","{'servicio_telefono': 'si', 'varias_lineas': '...","{'servicio_internet': 'fibra optica', 'segurid...","{'contrato': 'mensual', 'facturacion_electroni..."


In [5]:
#Observación general de los datos
datos_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7344 entries, 0 to 7343
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  7344 non-null   object
 1   Churn       7344 non-null   object
 2   cliente     7344 non-null   object
 3   telefono    7344 non-null   object
 4   internet    7344 non-null   object
 5   cuenta      7344 non-null   object
dtypes: object(6)
memory usage: 344.4+ KB


Se observa que existen columnas anidadas, por lo que es necesario realizar una normalización para obtener una mejor estructura del dataframe para el análisis y tratamiento de datos.

In [6]:
#se define una función que permite la extración de los datos anidados
def lectura_datos():
  global datos_churn

  with open("base_clientes.json", encoding='utf-8') as f:
    json_bruto = json.load(f)
  datos_churn = pd.json_normalize(json_bruto)

In [7]:
lectura_datos()

In [8]:
datos_churn.head(20)

,id_cliente,Churn,cliente.genero,cliente.anciano,cliente.pareja,cliente.dependientes,cliente.tiempo_servicio,telefono.servicio_telefono,telefono.varias_lineas,internet.servicio_internet,internet.seguridad_online,internet.backup_online,internet.proteccion_dispositivo,internet.soporte_tecnico,internet.tv_streaming,internet.peliculas_streaming,cuenta.contrato,cuenta.facturacion_electronica,cuenta.metodo_pago,cuenta.cobros.mensual,cuenta.cobros.Total
0,0002-ORFBO,no,femenino,0,si,si,9.0,si,no,DSL,no,si,no,si,si,no,None,None,None,NaN,None
1,0003-MKNFE,no,masculino,0,no,no,9.0,si,si,DSL,no,no,no,no,no,si,mensual,no,cheque,59.90,542.4
2,0004-TLHLJ,si,masculino,0,no,no,4.0,si,no,fibra optica,no,no,si,no,no,no,mensual,si,cheque electronico,73.90,280.85
3,0011-IGKFF,si,masculino,1,si,no,13.0,si,no,fibra optica,no,si,si,no,si,si,mensual,si,cheque electronico,98.00,1237.85
4,0013-EXCHZ,si,femenino,1,si,no,3.0,si,no,fibra optica,no,no,no,si,si,no,mensual,si,cheque,83.90,267.4
5,0013-MHZWF,no,femenino,0,no,si,9.0,si,no,DSL,no,no,no,si,si,si,mensual,si,tarjeta de credito (automatico),69.40,571.45
6,0013-SMEOE,no,femenino,1,si,no,71.0,si,no,fibra optica,si,si,si,si,si,si,dos años,si,transferencia bancaria (automatica),109.70,7904.25
7,0014-BMAQU,no,masculino,0,si,no,63.0,si,si,fibra optica,si,no,no,si,no,no,dos años,si,tarjeta de credito (automatico),84.65,5377.8
8,0015-UOCOJ,no,femenino,1,no,no,7.0,si,no,DSL,si,no,no,no,no,no,mensual,si,cheque electronico,48.20,340.35
9,0016-QLJIS,no,femenino,0,si,si,NaN,si,si,DSL,si,si,si,si,si,si,dos años,si,cheque,90.45,5957.9


## **2.2 Tratamiento de Datos**



### Revisión de Diccionario

In [9]:
#Se carga el diccionario con la información de los atributos
with open('/content/Diccionario.txt', 'r', encoding='utf-8') as file:
    contenido = file.read()
print(contenido)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Diccionario.txt'

### Análisis Exploratorio de los Datos

In [ ]:
#revisión general de los datos
datos_churn.info()

In [ ]:
datos_churn.describe()

In [ ]:
#Revisión de valores nulos en las columnas
datos_churn.isnull().sum()

In [ ]:
#Revisión de registros vacios en cada columna
registros_vacios = datos_churn.apply(lambda col: col.astype(str).str.strip().eq('').sum())
registros_vacios

Se observa que nuestra columna objetivo tiene 226 registros vacios

In [ ]:
#revisando los registros en de la columna Churn
datos_churn[datos_churn['Churn'] == '']

In [ ]:
#revisando los registros en de la columna cuenta.cobros.Total
datos_churn[datos_churn['cuenta.cobros.Total'] ==  ' ']

### Generación de la función Preprocesamiento

In [ ]:
def preprocesamiento():
  global datos_churn

  print(f'Cantidad de registros en el dataset: {datos_churn.shape[0]}')

  #identificación de índices con valores vacios en la columna 'cuenta.cobros.Total'
  idx = datos_churn[datos_churn['cuenta.cobros.Total'] ==  ' '].index
  #se llena los registros vacios identificados multiplicando los valores de 'cuenta.cobros.mensual'*24
  datos_churn.loc[idx, 'cuenta.cobros.Total'] = datos_churn.loc[idx, 'cuenta.cobros.mensual']*24
  #asignación a tiempo de servicio igual a 24 para aquellos indices identificados
  datos_churn.loc[idx, 'cliente.tiempo_servicio'] = datos_churn.loc[idx, 'cliente.tiempo_servicio'].replace(0, 24)

  #se convierte a columna numérica y convierte a nulo todo valor que no sea numérico
  datos_churn['cuenta.cobros.Total'] = pd.to_numeric(datos_churn['cuenta.cobros.Total'], errors='coerce')
  #se cambia el tipo de datos a float
  datos_churn['cuenta.cobros.Total'] = datos_churn['cuenta.cobros.Total'].astype('float64')

  #seleccion de columnas de tipo objeto
  columna_objects = datos_churn.select_dtypes(include='object').columns
  #se reemplaza los registros vacios en las columnas seleccionadas por nulos
  datos_churn[columna_objects] = datos_churn[columna_objects].replace('', np.nan)
  #se eliminan los valores nulos en la columnas object
  datos_churn.dropna(subset=columna_objects, inplace=True)
  #se eliminan duplicados
  datos_churn.drop_duplicates(inplace=True)
  print(f'Cantidad de registros en el dataset luego de eliminar nulos y duplicados en las columnas: {datos_churn.shape[0]}')

  #relleno los valores nulos de la columna 'cliente.tiempo_servicio'
  datos_churn['cliente.tiempo_servicio'] = datos_churn['cliente.tiempo_servicio'].fillna(datos_churn['cuenta.cobros.Total'] / datos_churn['cuenta.cobros.mensual'])
  #se reinicia el indice del dataframe
  datos_churn = datos_churn.reset_index(drop=True)

  #Cálculo del rango intercuartílico (IQR)
  valor = datos_churn['cliente.tiempo_servicio']
  Q1 = float(valor.quantile(.25))
  Q3 = float(valor.quantile(.75))
  IQR = Q3 - Q1
  #determinación de los limites
  limite_inferior = Q1 - (1.5*IQR)
  limite_superior = Q3 + (1.5*IQR)
  # Identificar índices con valores outliers donde 'cliente.tiempo_servicio' es menor que 'limite_inferior' o mayor que 'limite_superior'
  outlier_indices = datos_churn[(datos_churn['cliente.tiempo_servicio'] < limite_inferior) | (datos_churn['cliente.tiempo_servicio'] > limite_superior)].index

  #Se recalcula el valor de 'cliente.tiempo_servicio' en los indices con outliers identificados dividiendo cuenta.cobros.Total por cuenta.cobros.mensual
  datos_churn.loc[outlier_indices, 'cliente.tiempo_servicio'] = datos_churn['cuenta.cobros.Total'] / datos_churn['cuenta.cobros.mensual']
  #Se vuelve a calcular el IQR luego de la corrección
  valor = datos_churn['cliente.tiempo_servicio']
  Q1 = valor.quantile(.25)
  Q3 = valor.quantile(.75)
  IQR = Q3 - Q1
  #se determina nuevos limites luego del recalculo
  limite_inferior = Q1 - (1.5*IQR)
  limite_superior = Q3 + (1.5*IQR)
  #se elimina los outliers del dataframe
  datos_churn = datos_churn[(datos_churn['cliente.tiempo_servicio'] >= limite_inferior) & (datos_churn['cliente.tiempo_servicio'] <= limite_superior)]
  #se reinicia el indice del dataframe
  datos_churn = datos_churn.reset_index(drop=True)

  print(f'Cantidad de registros en el dataset luego de eliminar registros con outliers en tiempo de servicio: {datos_churn.shape[0]}')


In [ ]:
preprocesamiento()

In [ ]:
datos_churn.info()

## **3. Normalización de Datos**

Observación de valores únicos

In [ ]:
#Se analiza que valores únicos contiene cada columna
for col in datos_churn.columns:
    print(f"Columna: {col}")
    print(datos_churn[col].unique())
    print("-" * 30)



En la normalización se busca eliminar columnas innecesarias y se reemplazan los valores categóricos con representaciones numéric

In [ ]:
#Se define una función normalización
def normalización():
    global datos_churn

    # Eliminar columna 'id_cliente'
    datos_churn = datos_churn.drop(columns=['id_cliente'])

    # Diccionario de mapeo para las categorías
    mapeo = {
        'no': 0,
        'si': 1,
        'masculino': 0,
        'femenino': 1
    }

    # Columnas a normalizar
    columnas_a_normalizar = [
        'telefono.servicio_telefono', 'Churn',
        'cliente.pareja', 'cliente.dependientes',
        'cuenta.facturacion_electronica', 'cliente.genero'
    ]

    # Reemplazo de valores categóricos por numéricos
    datos_churn[columnas_a_normalizar] = datos_churn[columnas_a_normalizar].replace(mapeo)

    # Creación de variables dummy para columnas categóricas restantes
    datos_churn = pd.get_dummies(datos_churn, drop_first=True)

    # Convertir valores booleanos generados por dummies a numéricos
    cols_booleanas = datos_churn.select_dtypes(include='bool').columns
    datos_churn[cols_booleanas] = datos_churn[cols_booleanas].astype(int)

    # Reinicio del índice
    datos_churn.reset_index(drop=True, inplace=True)

    return datos_churn

In [ ]:
normalización()
datos_churn

In [ ]:
#se observa nuevamente como queda el dataframe
datos_churn.info()

### **3.1. Balanceamiento de los datos**

In [ ]:
#se verifica visualmente como se dividen los valores de la variable 'Churn'
ax = sns.countplot(x='Churn', data=datos_churn)

In [ ]:
#se observa la distribución de forma numérica
datos_churn.Churn.value_counts()

In [ ]:
#dividiendo los datos en características y target
X = datos_churn.drop('Churn', axis = 1)
y = datos_churn['Churn']

In [ ]:
smt = SMOTE(random_state=123)
X, y = smt.fit_resample(X, y)

In [ ]:
#unión de los datos balanceados
datos_churn = pd.concat([X, y], axis=1)

In [ ]:
#verificación 2 - balanceamiento
ax = sns.countplot(x='Churn', data=datos_churn)

In [ ]:
datos_churn.Churn.value_counts()

### **3.2. Eliminación de Columnas innecesarias**

Mediante distintos métodos de análisis gráficos se analiza que columnas son irrelevantes para realizar la predicción

In [ ]:
#se define una función que permita generar un diagrama de puntos sin tomar encuenta nuestra columna objetivo 'Churn'
def diagrama_puntos(df, inicio, fin):
  df = df[df.select_dtypes(include= 'int').columns]
  y = df['Churn']
  x = df.drop(columns='Churn')
  df = pd.concat([y,x.iloc[:,inicio:fin]], axis=1)
  df_melted = pd.melt(df, id_vars='Churn', var_name='features', value_name='valores')
  plt.figure(figsize=(12,6))
  sns.pointplot(x='features', y='valores', hue='Churn', data=df_melted)
  plt.xticks(rotation=90)
  plt.show()

In [ ]:
diagrama_puntos(datos_churn, 0, 31)

In [ ]:
#Se define un mapa de calor para observar la correlación entre las columnas, tomando como referencia principal la correlación con 'Churn'
def mapa_calor(df):
  grafico = df.corr()
  plt.figure(figsize=(12,6))
  sns.heatmap(grafico, annot=True, fmt='.1f')
  return grafico

In [ ]:
mapa_calor(datos_churn)

#### Interpretación

Se observa que las columnas: 'cliente.anciano','telefono.servicio_telefono', 'telefono.varias_lineas_sin servicio de telefono','telefono.varias_lineas_si' tienen poca o nula influencia para establecer una predicción con base a dichas columnas

In [ ]:
# Se eliminan las columnas innecesarias
columnas_a_eliminar = ['cliente.anciano','telefono.servicio_telefono', 'telefono.varias_lineas_sin servicio de telefono','telefono.varias_lineas_si']
datos_churn = datos_churn.drop(columns=columnas_a_eliminar)
datos_churn

## 4. Modelado

### **4.1 Modelo RFE**

In [ ]:
# Se divide nuestro conjunto de datos para establecer la parte de entrenamiento y prueba
y = datos_churn['Churn']
x = datos_churn.drop(columns='Churn')
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state = 50)

In [ ]:
# se define una función que permite ejecutar la predicción con base a el modelo de clasificación RFE
def pronosticar_RFE(train_x, train_y, test_x, test_y):
  model = RandomForestClassifier()
  selector = RFE(estimator=model, n_features_to_select=10, step=1)
  train_x_selected = selector.fit_transform(train_x, train_y)
  test_x_selected = selector.transform(test_x)
  model.fit(train_x_selected, train_y)
  test_score = model.score(test_x_selected, test_y)
  print(f'Accuracy en prueba con RFE: {test_score}')

In [ ]:
#Se ejecuta la función con el modelo
pronosticar_RFE(train_x, train_y, test_x, test_y)